In [1]:
from weiner_variation.sim.data_structures import DrawInput
from weiner_variation.sim.process import PASS_SEQUENCE, DIAMETER, TEMPERATURE, DIAMETER_STD, TEMPERATURE_STD, IN_PROFILE
from weiner_variation.sim.config import SAMPLE_COUNT, FIELDS, SEED
from weiner_variation.config import DATA_DIR

import pandas as pd
import numpy as np
from scipy.stats import norm
from copy import deepcopy
import tqdm
from multiprocessing import Pool
import os

In [2]:
import pyroll.basic as pr
import pyroll.jmak

In [3]:
OUTPUT_FILENAME = "sim_nominal_results.csv"
REPORT_FILENAME = "nominal_report.html"

In [4]:
sequence = deepcopy(PASS_SEQUENCE)
sequence.solve(IN_PROFILE)       

result = { ("draw", "diameter"): DIAMETER, ("draw", "temperature"): TEMPERATURE } | (
    FIELDS.items()
    |> starmap$((key, extractor) -> (
            sequence.units 
            |> filter$(u -> isinstance(u, pr.RollPass)) 
            |> map$(u -> ((key, u.label), extractor(u))) 
            |> filter$(t -> t[1] is not None)
        )
    )
    |> flatten
    |> dict            
)

/home/maxwe/Git/publication-weiner-variation/.venv_sim/lib/python3.11/site-packages/pyroll/jmak/jmak.py:364: RuntimeWarning: invalid value encountered in scalar power
  (d_rx ** self.in_profile.jmak_parameters.s


RuntimeError: Solution of sub units failed at unit Transport 'R1-R2'.

In [ ]:
df = result |> pd.Series |> .T |> .infer_objects()
df

In [ ]:
df.to_csv(DATA_DIR / OUTPUT_FILENAME)

In [ ]:
from pyroll.report import report
f = DATA_DIR / REPORT_FILENAME
f.write_text(report(sequence))